In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 1: MOUNT GOOGLE DRIVE                                  ║
# ╚══════════════════════════════════════════════════════════════╝

from google.colab import drive
drive.mount('/content/drive')

import os
CHECKPOINT_DIR = '/content/drive/MyDrive/PromptPAR_checkpoints/Vit_Market1501'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs('/data/jinjiandong', exist_ok=True)

print(f" Checkpoint dir: {CHECKPOINT_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Checkpoint dir: /content/drive/MyDrive/PromptPAR_checkpoints/Vit_Market1501


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 3: CLONE REPOSITORIES                                  ║
# ╚══════════════════════════════════════════════════════════════╝

import os
import shutil

%cd /content

# Clone OpenPAR
if os.path.exists('/content/OpenPAR'):
    shutil.rmtree('/content/OpenPAR')
! git clone https://github.com/Event-AHU/OpenPAR.git
print(" Cloned OpenPAR")

# Clone Market-1501 Attributes
if os.path.exists('/content/Market-1501_Attribute'):
    shutil.rmtree('/content/Market-1501_Attribute')
!git clone https://github.com/vana77/Market-1501_Attribute.git
print(" Cloned Market-1501_Attribute")

# Verify
!ls -la /content/

/content
Cloning into 'OpenPAR'...
remote: Enumerating objects: 2108, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 2108 (delta 331), reused 248 (delta 248), pack-reused 1728 (from 1)
Receiving objects: 100% (2108/2108), 80.72 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.
 Cloned OpenPAR
Cloning into 'Market-1501_Attribute'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 109 (delta 0), reused 0 (delta 0), pack-reused 106 (from 1)
Receiving objects: 100% (109/109), 138.96 KiB | 5.34 MiB/s, done.
Resolving deltas: 100% (44/44), done.
 Cloned Market-1501_Attribute
total 32
drwxr-xr-x  1 root root 4096 Dec 10 02:50 .
drwxr-xr-x  1 root root 4096 Dec 10 02:26 ..
drwxr-xr-x  4 root root 4096 Nov 20 14:30 .config
drwxr-xr-x  4 root root 4096 Dec 10 02:49 data
drwx------  5 root root 4096 Dec 10 02:26 d

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 4: CÀI ĐẶT DEPENDENCIES                                ║
# ╚══════════════════════════════════════════════════════════════╝

%cd /content/OpenPAR/PromptPAR

! pip install -q ftfy regex tqdm easydict scipy
!pip install -q git+https://github.com/openai/CLIP.git

print("\n Dependencies installed")

/content/OpenPAR/PromptPAR


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 5: TẢI MARKET-1501 DATASET                             ║
# ╚══════════════════════════════════════════════════════════════╝

import os

os.makedirs('/content/data', exist_ok=True)
%cd /content/data

# Tải Market-1501
if not os.path. exists('/content/data/Market-1501-v15.09.15'):
    print(" Đang tải Market-1501...")
    !pip install -q gdown
    !gdown "https://drive.google.com/uc?id=0B8-rUzbwVRk0c054eEozWG9COHM" -O Market-1501-v15.09.15.zip --fuzzy
    !unzip -q Market-1501-v15.09.15.zip
    !rm Market-1501-v15.09.15. zip
    print(" Đã tải và giải nén Market-1501")
else:
    print(" Market-1501 đã tồn tại")

! ls -la /content/data/Market-1501-v15.09.15/

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 6: TẢI VIT PRETRAINED MODEL                            ║
# ╚══════════════════════════════════════════════════════════════╝

import os

os.makedirs('/data/jinjiandong', exist_ok=True)

vit_path = '/data/jinjiandong/jx_vit_base_p16_224-80ecf9dd.pth'

if not os.path.exists(vit_path):
    print("🔄 Đang tải ViT pretrained...")
    !wget -q https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth -O {vit_path}

if os.path.exists(vit_path):
    size = os.path.getsize(vit_path) / 1024 / 1024
    print(f" ViT pretrained: {size:.1f} MB")
else:
    print(" Tải ViT thất bại!")

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 7: SỬA CODE PROMPTPAR ĐỂ HỖ TRỢ MARKET1501             ║
# ╚══════════════════════════════════════════════════════════════╝

import os
import shutil

# Clone lại để đảm bảo code sạch
%cd /content
if os.path.exists('/content/OpenPAR'):
    shutil.rmtree('/content/OpenPAR')
! git clone -q https://github.com/Event-AHU/OpenPAR.git
print("Cloned fresh OpenPAR")

# ========== 1. SỬA clip/model.py ==========
model_py = '/content/OpenPAR/PromptPAR/clip/model.py'

with open(model_py, 'r') as f:
    lines = f.readlines()

new_lines = []
for line in lines:
    # Sửa dòng assert
    if "assert args.dataset in [" in line and "Market1501" not in line:
        new_lines.append("assert args.dataset in ['PA100k', 'RAPV1','RAPV2','PETA','WIDER','PETAzs','RAPzs','UPAR','YCJC','RAPV1Expand','Market1501'], \\\n")
    # Sửa dòng datasets_attrnum
    elif "datasets_attrnum=" in line and "Market1501" not in line:
        new_lines. append("datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':40,'RAPzs':53,'RAPV2':54,'WIDER':14,'RAPV1Expand':51,'Market1501':27}\n")
    else:
        new_lines.append(line)

with open(model_py, 'w') as f:
    f. writelines(new_lines)

print(" Đã sửa clip/model.py")

# ========== 2. SỬA dataset/AttrDataset.py ==========
attr_py = '/content/OpenPAR/PromptPAR/dataset/AttrDataset.py'

with open(attr_py, 'r') as f:
    content = f.read()

# Thêm Market1501
content = content.replace(
    "['PA100k', 'RAPV1','RAPV2','PETA','WIDER','RAPzs','PETAzs','UPAR','YCJC',]",
    "['PA100k', 'RAPV1','RAPV2','PETA','WIDER','RAPzs','PETAzs','UPAR','YCJC','Market1501']"
)

# Sửa đường dẫn dataset
content = content. replace(
    "dataset_dir='/data/jinjiandong/datasets/'",
    "dataset_dir='/content/data/'"
)

with open(attr_py, 'w') as f:
    f.write(content)

print(" Đã sửa dataset/AttrDataset.py")

# ========== VERIFY ==========
print("\n Verify model.py:")
with open(model_py, 'r') as f:
    for i, line in enumerate(f, 1):
        if 'datasets_attrnum' in line:
            print(f"   Line {i}: {line.strip()[:70]}...")
            if "'Market1501':27" in line:
                print("    Market1501 = 27 attributes")
            else:
                print("    Chưa đúng!")
            break

/content
Cloned fresh OpenPAR
 Đã sửa clip/model.py
 Đã sửa dataset/AttrDataset.py

 Verify model.py:
   Line 17: datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':...
    Market1501 = 27 attributes


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 7 FIX: SỬA TRỰC TIẾP FILE MODEL.PY                     ║
# ╚══════════════════════════════════════════════════════════════╝

model_py = '/content/OpenPAR/PromptPAR/clip/model.py'

with open(model_py, 'r') as f:
    content = f.read()

# Hiển thị trước khi sửa
print(" TRƯỚC KHI SỬA:")
for line in content.split('\n')[13:19]:
    print(f"   {line[:80]}")

# Sửa tất cả các pattern có thể
replacements = [
    # Pattern 1
    ("datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':40,'RAPzs':53,'RAPV2':54,'WIDER':14,\"RAPV1Expand\":51}",
     "datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':40,'RAPzs':53,'RAPV2':54,'WIDER':14,'RAPV1Expand':51,'Market1501':27}"),
    # Pattern 2 (nếu đã có Market1501 nhưng sai số)
    ("'Market1501':39", "'Market1501':27"),
    # Assert pattern
    ("['PA100k', 'RAPV1','RAPV2','PETA','WIDER','PETAzs','RAPzs','UPAR','YCJC',\"RAPV1Expand\"]",
     "['PA100k', 'RAPV1','RAPV2','PETA','WIDER','PETAzs','RAPzs','UPAR','YCJC','RAPV1Expand','Market1501']"),
]

for old, new in replacements:
    if old in content:
        content = content. replace(old, new)
        print(f"\n Đã thay thế pattern")

with open(model_py, 'w') as f:
    f.write(content)

# Hiển thị sau khi sửa
print("\n SAU KHI SỬA:")
with open(model_py, 'r') as f:
    content = f.read()
for line in content.split('\n')[13:19]:
    print(f"   {line[:80]}")

# Verify
if "'Market1501':27" in content:
    print("\n OK!  Market1501 = 27")
else:
    print("\n Chưa sửa được!  Cần sửa thủ công")

 TRƯỚC KHI SỬA:
   args = parser.parse_args()
   assert args.dataset in ['PA100k', 'RAPV1','RAPV2','PETA','WIDER','PETAzs','RAPzs
       f'dataset name {args.dataset} is not exist,The legal name is PA100k,RAPV1,RA
   datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':40,'RAPzs'
   attr_num=datasets_attrnum[args.dataset]
   class Bottleneck(nn.Module):

 SAU KHI SỬA:
   args = parser.parse_args()
   assert args.dataset in ['PA100k', 'RAPV1','RAPV2','PETA','WIDER','PETAzs','RAPzs
       f'dataset name {args.dataset} is not exist,The legal name is PA100k,RAPV1,RA
   datasets_attrnum={'PA100k':26,'RAPV1':51,'PETA':35,'PETAzs':35,'UPAR':40,'RAPzs'
   attr_num=datasets_attrnum[args.dataset]
   class Bottleneck(nn.Module):

 OK!  Market1501 = 27


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 8: TẠO DATASET PICKLE CHO MARKET1501                   ║
# ╚══════════════════════════════════════════════════════════════╝

import scipy.io
import numpy as np
import os
import pickle
import shutil
from easydict import EasyDict

# ========== PATHS ==========
MARKET_DIR = '/content/data/Market-1501-v15.09.15'
ATTR_FILE = '/content/Market-1501_Attribute/market_attribute.mat'
SAVE_DIR = '/content/data/Market1501'

# ========== CHECK FILES ==========
print("Kiểm tra files...")
if not os.path. exists(MARKET_DIR):
    raise Exception("Chưa có Market-1501!  Chạy Cell 5 trước.")
if not os.path. exists(ATTR_FILE):
    raise Exception("Chưa có market_attribute.mat! Chạy Cell 3 trước.")
print("Files OK")

# ========== LOAD . MAT FILE ==========
mat_data = scipy. io.loadmat(ATTR_FILE)
market_attr = mat_data['market_attribute'][0][0]
train_attr_mat = market_attr['train'][0][0]
test_attr_mat = market_attr['test'][0][0]

# Lấy tên attributes TỪ FILE . MAT (27 attributes)
attr_names = [name for name in train_attr_mat.dtype. names if name != 'image_index']
print(f"📊 Attributes từ . mat: {len(attr_names)}")

# ========== GET IMAGE LISTS ==========
train_dir = os.path. join(MARKET_DIR, 'bounding_box_train')
test_dir = os.path. join(MARKET_DIR, 'bounding_box_test')

train_images = sorted([f for f in os. listdir(train_dir)
                      if f.endswith('.jpg') and not f.startswith('-1')])
test_images = sorted([f for f in os. listdir(test_dir)
                     if f.endswith('.jpg') and not f.startswith('-1') and not f.startswith('0000')])

print(f"Train images: {len(train_images)}")
print(f"Test images: {len(test_images)}")

# ========== CREATE DATASET ==========
image_name_list = []
label_list = []
train_idx = []
test_idx = []
idx = 0

# Process TRAINING images
print("\n Processing training images...")
for img_name in train_images:
    try:
        identity_id = int(img_name.split('_')[0])
        if 1 <= identity_id <= 751:
            image_name_list.append(os.path.join('bounding_box_train', img_name))
            attr_idx = identity_id - 1
            label = [int(train_attr_mat[attr_name][0][attr_idx]) - 1
                     if int(train_attr_mat[attr_name][0][attr_idx]) > 0 else 0
                     for attr_name in attr_names]
            label_list.append(label)
            train_idx.append(idx)
            idx += 1
    except:
        continue
print(f" Train samples: {len(train_idx)}")

# Process TEST images
print(" Processing test images...")
for img_name in test_images:
    try:
        identity_id = int(img_name.split('_')[0])
        if 1 <= identity_id <= 750:
            image_name_list.append(os.path. join('bounding_box_test', img_name))
            attr_idx = identity_id - 1
            label = [int(test_attr_mat[attr_name][0][attr_idx]) - 1
                     if int(test_attr_mat[attr_name][0][attr_idx]) > 0 else 0
                     for attr_name in attr_names]
            label_list. append(label)
            test_idx. append(idx)
            idx += 1
    except:
        continue
print(f" Test samples: {len(test_idx)}")

# ========== CREATE EASYDICT ==========
dataset = EasyDict()
dataset.description = 'market1501'
dataset.root = SAVE_DIR
dataset. image_name = image_name_list
dataset.label = np.array(label_list, dtype=np.float32)
dataset. attributes = attr_names  # 27 attributes từ .mat

dataset.partition = EasyDict()
dataset.partition. train = np.array(train_idx, dtype=np.int64)
dataset.partition.val = np.array([], dtype=np.int64)
dataset. partition.test = np.array(test_idx, dtype=np.int64)
dataset.partition.trainval = np.array(train_idx, dtype=np.int64)

# Weights
train_labels = dataset.label[dataset. partition.train]
label_ratio = np.clip(np.mean(train_labels, axis=0), 0.01, 0.99)
dataset.weight_train = np.exp(-label_ratio). astype(np. float32)
dataset.weight_trainval = dataset.weight_train. copy()

# ========== SAVE ==========
if os. path.exists(SAVE_DIR):
    shutil.rmtree(SAVE_DIR)
os.makedirs(SAVE_DIR)

pkl_path = os.path. join(SAVE_DIR, 'pad. pkl')
with open(pkl_path, 'wb') as f:
    pickle.dump(dataset, f)
print(f"\n Saved: {pkl_path}")

os.symlink(train_dir, os. path.join(SAVE_DIR, 'bounding_box_train'))
os.symlink(test_dir, os. path.join(SAVE_DIR, 'bounding_box_test'))
print("Created symlinks")

# ========== VERIFY ==========
print("\n" + "=" * 60)
print(" DATASET SUMMARY")
print("=" * 60)
print(f"Labels shape: {dataset. label.shape}")
print(f"Attributes: {len(dataset. attributes)}")
print(f"Train: {len(dataset.partition. train)}")
print(f"Test: {len(dataset.partition. test)}")

# Verify file exists
if os.path.exists(pkl_path):
    size = os.path. getsize(pkl_path) / 1024 / 1024
    print(f"pad.pkl: {size:.2f} MB")

if len(dataset.attributes) == dataset.label.shape[1]:
    print(f"\n OK!  attributes = labels = {len(dataset.attributes)}")
else:
    print(f"\n MISMATCH!")

Kiểm tra files...
Files OK
📊 Attributes từ . mat: 27
Train images: 12936
Test images: 13115

 Processing training images...
 Train samples: 7421
 Processing test images...
 Test samples: 6868

 Saved: /content/data/Market1501/pad. pkl
Created symlinks

 DATASET SUMMARY
Labels shape: (14289, 27)
Attributes: 27
Train: 7421
Test: 6868
pad.pkl: 2.25 MB

 OK!  attributes = labels = 27


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  TẠO PAD.PKL                                                 ║
# ╚══════════════════════════════════════════════════════════════╝

import scipy.io
import numpy as np
import os
import pickle
import shutil
from easydict import EasyDict

# ========== PATHS ==========
MARKET_DIR = '/content/data/Market-1501-v15.09.15'
ATTR_FILE = '/content/Market-1501_Attribute/market_attribute.mat'
SAVE_DIR = '/content/data/Market1501'

# Xóa và tạo lại thư mục
if os.path.exists(SAVE_DIR):
    shutil.rmtree(SAVE_DIR)
os.makedirs(SAVE_DIR)

# ========== LOAD . MAT FILE ==========
mat_data = scipy.io. loadmat(ATTR_FILE)
market_attr = mat_data['market_attribute'][0][0]
train_attr_mat = market_attr['train'][0][0]
test_attr_mat = market_attr['test'][0][0]

# Lấy 27 attributes từ . mat
attr_names = [name for name in train_attr_mat.dtype.names if name != 'image_index']
print(f"Attributes: {len(attr_names)}")

# ========== GET IMAGE LISTS ==========
train_dir = os.path. join(MARKET_DIR, 'bounding_box_train')
test_dir = os.path. join(MARKET_DIR, 'bounding_box_test')

train_images = sorted([f for f in os.listdir(train_dir)
                      if f.endswith('.jpg') and not f.startswith('-1')])
test_images = sorted([f for f in os. listdir(test_dir)
                     if f.endswith('.jpg') and not f.startswith('-1') and not f.startswith('0000')])

print(f"Train: {len(train_images)}, Test: {len(test_images)}")

# ========== CREATE DATASET ==========
image_name_list = []
label_list = []
train_idx = []
test_idx = []
idx = 0

# TRAINING
for img_name in train_images:
    try:
        identity_id = int(img_name.split('_')[0])
        if 1 <= identity_id <= 751:
            image_name_list.append(os.path.join('bounding_box_train', img_name))
            attr_idx = identity_id - 1
            label = [max(0, int(train_attr_mat[attr_name][0][attr_idx]) - 1) for attr_name in attr_names]
            label_list.append(label)
            train_idx.append(idx)
            idx += 1
    except:
        continue

# TEST
for img_name in test_images:
    try:
        identity_id = int(img_name.split('_')[0])
        if 1 <= identity_id <= 750:
            image_name_list.append(os.path. join('bounding_box_test', img_name))
            attr_idx = identity_id - 1
            label = [max(0, int(test_attr_mat[attr_name][0][attr_idx]) - 1) for attr_name in attr_names]
            label_list.append(label)
            test_idx.append(idx)
            idx += 1
    except:
        continue

print(f" Train: {len(train_idx)}, Test: {len(test_idx)}")

# ========== CREATE EASYDICT ==========
dataset = EasyDict()
dataset.description = 'market1501'
dataset.root = SAVE_DIR
dataset. image_name = image_name_list
dataset.label = np.array(label_list, dtype=np.float32)
dataset. attributes = attr_names

dataset.partition = EasyDict()
dataset.partition.train = np.array(train_idx, dtype=np.int64)
dataset.partition.val = np.array([], dtype=np.int64)
dataset. partition.test = np.array(test_idx, dtype=np.int64)
dataset.partition.trainval = np.array(train_idx, dtype=np.int64)

# Weights
train_labels = dataset.label[dataset.partition.train]
label_ratio = np.clip(np.mean(train_labels, axis=0), 0.01, 0.99)
dataset.weight_train = np.exp(-label_ratio). astype(np.float32)
dataset.weight_trainval = dataset.weight_train. copy()

# ========== SAVE PICKLE ==========
pkl_path = os. path.join(SAVE_DIR, 'pad.pkl')
with open(pkl_path, 'wb') as f:
    pickle.dump(dataset, f)

# ========== CREATE SYMLINKS ==========
os.symlink(train_dir, os. path.join(SAVE_DIR, 'bounding_box_train'))
os.symlink(test_dir, os. path.join(SAVE_DIR, 'bounding_box_test'))

# ========== VERIFY ==========
print("\n" + "=" * 60)
if os.path.exists(pkl_path):
    size = os.path. getsize(pkl_path) / 1024 / 1024
    print(f"CREATED: {pkl_path} ({size:.2f} MB)")
    print(f"Labels: {dataset.label.shape}")
    print(f"Attributes: {len(dataset.attributes)}")
    print(f"\n THÀNH CÔNG! Chạy Cell 10 để training")
else:
    print("THẤT BẠI!")

Attributes: 27
Train: 12936, Test: 13115
 Train: 7421, Test: 6868

CREATED: /content/data/Market1501/pad.pkl (2.25 MB)
Labels: (14289, 27)
Attributes: 27

 THÀNH CÔNG! Chạy Cell 10 để training


In [ ]:
# --- PATCH: lưu checkpoint thẳng vào Drive, không dùng timestamp ---

train_py = "/content/OpenPAR/PromptPAR/train.py"

with open(train_py, "r") as f:
    code = f.read()

# Block cũ dùng timestamp
old = """start_time=time_str()
    print(f'start_time is {start_time}')
    log_dir = os.path.join('logs', args.dataset)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_dir = os.path.join(log_dir, start_time)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)"""

# Block mới: lấy đường dẫn từ --dir
new = """log_dir = args.dir
    os.makedirs(log_dir, exist_ok=True)"""

# Thay thế
code = code.replace(old, new)

with open(train_py, "w") as f:
    f.write(code)

print("train.py đã được chỉnh để lưu vào Drive")


train.py đã được chỉnh để lưu vào Drive


In [ ]:
# PATCH: thêm resume logic vào train.py (ngắn gọn)

train_py = "/content/OpenPAR/PromptPAR/train.py"

with open(train_py, "r") as f:
    code = f.read()

if "torch.load" not in code:
    print("Đang thêm logic resume...")

    code = code.replace(
        "model = TransformerClassifier(clip_model,train_set.attr_num,train_set.attributes)",
        """model = TransformerClassifier(clip_model,train_set.attr_num,train_set.attributes)

    # ---- AUTO RESUME ----
    start_epoch = 1
    if args.checkpoint:
        import glob
        ckpts = glob.glob(os.path.join(args.dir, "epoch*.pth"))
        if ckpts:
            latest = max(ckpts, key=os.path.getctime)
            print("Resume từ:", latest)
            ckpt = torch.load(latest, map_location=device)
            model.load_state_dict(ckpt["model_state_dict"])
            clip_model.load_state_dict(ckpt["clip_model"])
            try:
                optimizer.load_state_dict(ckpt["optimizer"])
            except:
                pass
            start_epoch = ckpt["epoch"] + 1
    """
    )

    code = code.replace(
        "for i in range(1, epoch+1):",
        "for i in range(start_epoch, epoch+1):"
    )

    with open(train_py, "w") as f:
        f.write(code)

    print("Patch xong")
else:
    print("File đã có logic resume, bỏ qua")


Đang thêm logic resume...
Patch xong


In [ ]:
# === FIX PyTorch 2.6+ checkpoint loading on Colab ===
from easydict import EasyDict
import torch.serialization

torch.serialization.add_safe_globals([EasyDict])
print("✅ Patched: allow EasyDict in torch.load")


✅ Patched: allow EasyDict in torch.load


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 10: TRAINING (LƯU THẲNG VÀO DRIVE)                     ║
# ╚══════════════════════════════════════════════════════════════╝

%cd /content/OpenPAR/PromptPAR

import os, glob

# Tạo thư mục trong Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/PromptPAR_checkpoints/Vit_Market1501"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Tạo symlink từ local output sang Drive
LOCAL_OUTPUT = "/content/OpenPAR/PromptPAR/exp_result"
if os.path.exists(LOCAL_OUTPUT):
    if os.path.islink(LOCAL_OUTPUT):
        os.unlink(LOCAL_OUTPUT) # Remove symbolic link directly
    elif os.path.isdir(LOCAL_OUTPUT):
        import shutil
        shutil.rmtree(LOCAL_OUTPUT) # Remove directory
os.symlink(CHECKPOINT_DIR, LOCAL_OUTPUT)

# Tự động tìm checkpoint mới nhất để resume
checkpoint_files = glob.glob(f"{CHECKPOINT_DIR}/*.pth")
resume_arg = ""
if checkpoint_files:
    latest_ckpt = max(checkpoint_files, key=os.path.getctime)
    print(f"Tìm thấy checkpoint mới nhất → tiếp tục từ: {os.path.basename(latest_ckpt)}")
    resume_arg = f"--checkpoint"
else:
    print("Không có checkpoint → train từ đầu")

print(f" Symlink: {LOCAL_OUTPUT} → {CHECKPOINT_DIR}")
print(" Bắt đầu training.. .\n")

cmd = f"""
python train.py Market1501 \
--batchsize 32 \
--epoch 60 \
--height 224 \
--width 224 \
--lr 5e-5 \
--weight_decay 1e-4 \
--clip_lr 1e-5 \
--clip_weight_decay 1e-4 \
--text_prompt 3 \
--vis_prompt 20 \
--vis_depth 12 \
--div_num 4 \
--overlap_row 2 \
--mm_layers 1 \
--smooth_param 0.1 \
--ag_threshold 0.5 \
--train_split trainval \
--valid_split test \
--save_freq 5 \
--use_div \
--use_textprompt \
--use_mm_former \
--use_GL \
--dir {CHECKPOINT_DIR} \
{resume_arg}
"""

print(cmd)
!{cmd}


/content/OpenPAR/PromptPAR
Tìm thấy checkpoint mới nhất → tiếp tục từ: epoch5.pth
 Symlink: /content/OpenPAR/PromptPAR/exp_result → /content/drive/MyDrive/PromptPAR_checkpoints/Vit_Market1501
 Bắt đầu training.. .


python train.py Market1501 --batchsize 32 --epoch 60 --height 224 --width 224 --lr 5e-5 --weight_decay 1e-4 --clip_lr 1e-5 --clip_weight_decay 1e-4 --text_prompt 3 --vis_prompt 20 --vis_depth 12 --div_num 4 --overlap_row 2 --mm_layers 1 --smooth_param 0.1 --ag_threshold 0.5 --train_split trainval --valid_split test --save_freq 5 --use_div --use_textprompt --use_mm_former --use_GL --dir /content/drive/MyDrive/PromptPAR_checkpoints/Vit_Market1501 --checkpoint

redirector stdout
OrderedDict([('dataset', 'Market1501'),
             ('batchsize', 32),
             ('epoch', 60),
             ('height', 224),
             ('width', 224),
             ('lr', 5e-05),
             ('weight_decay', 0.0001),
             ('ag_threshold', 0.5),
             ('smooth_param', 0.1),
     